In [26]:
# -*- coding: utf-8 -*-
"""
Created on Fri Sep 24 23:54:22 2021

@author: VONO
"""

import pandas as pd
import numpy as np

In [27]:
#导入数据
stock=pd.read_csv(r"\学习资料\实证金融学\data_replication\stock1.csv")

In [28]:
#计算超额收益率并设置时间作为索引
#stock=stock.drop(stock[(stock.RET=='C') | (stock.RET=='B')].index)
stock['RET'].replace(['B','C'],[np.nan,np.nan],inplace=True)
stock['RET']=stock['RET'].apply(float)
stock['ret']=stock['RET'] - stock['rf']
stock['mr']=stock['vwretd'] - stock['rf']
stock['id']=stock['PERMNO']
stock=stock[['id','date','ret','mr']]
stock.set_index(pd.to_datetime(stock['date'].astype(str)),inplace=True)
stock.drop(columns=['date'],inplace=True)

#计算个股的rolling volatility
stock.sort_values(by=['id','date'],inplace=True)
stock['lnret']=np.log(1+stock['ret'])
stock['rstd']=stock.groupby(stock['id'])['lnret'].apply(lambda x: x.rolling('365D',min_periods=120,closed='left').std())
stock.drop(columns=['lnret'],inplace=True)

#计算市场的rolling volatility
market=stock[['mr']]
market = market.reset_index()
market = market.drop_duplicates(subset=["date","mr"])
market = market.set_index("date")
mstd=np.log(1+market['mr']).rolling('365D',min_periods=120,closed='left').std()
mstd=pd.DataFrame(mstd)
mstd.rename(columns={'mr':'mstd'},inplace=True)

#合并数据
stock=pd.merge(stock,mstd,on=['date'],how='left')
del market
del mstd

#计算相关系数
#首先计算个股加和回报率
stock.sort_values(by=['id','date'],inplace=True)
stock['lnret1']=(stock['ret']+1).apply(np.log)
stock['lnret']=stock['lnret1']+stock.groupby(stock['id'])['lnret1'].shift(1)+stock.groupby(stock['id'])['lnret1'].shift(2)
stock.drop(columns=['lnret1'],inplace=True)
#计算市场加和回报率
stock['lnmr1']=(stock['mr']+1).apply(np.log)
stock['lnmr']=stock['lnmr1']+stock.groupby(stock['id'])['lnmr1'].shift(1)+stock.groupby(stock['id'])['lnmr1'].shift(2)
stock.drop(columns=['lnmr1'],inplace=True)
#计算相关系数
corr=stock.groupby(stock['id'])[['lnret','lnmr']].apply(lambda x: x.rolling('1826D',min_periods=750,closed='left').corr())
corr=corr.loc[(slice(None),slice(None),'lnret'),'lnmr']
corr=pd.DataFrame(corr)
corr.rename(columns={'lnmr':'corr'},inplace=True)

#再次合并数据
stock=pd.merge(stock,corr,on=['id','date'],how='left')
stock['beta']=stock['corr']*stock['rstd']/stock['mstd']

In [29]:
stock.to_csv(r"D:\学习资料\实证金融学\data_replication\stock1_ex_ante_beta.csv")